In [1]:
from matplotlib import pyplot
import numpy
%matplotlib inline
from matplotlib import rcParams
rcParams['font.family'] = 'serif'
rcParams['font.size'] = 16

In [6]:
#Parameters
nx = 41
ny = 41

l = 1.
h = 1.

xmin = 0
xmax = 1.
ymin = -0.5
ymax = 0.5

u = numpy.ones(nx)

dx = l/(nx-1)
dy = h/(ny-1)

#Mesh
x  = numpy.linspace(0,xmax,nx)
y  = numpy.linspace(ymin,ymax,ny)
X,Y = numpy.meshgrid(x,y)

#Initial conditions
psi = numpy.zeros((ny, nx)) #Create a XxY vector of zeros for psi
w = numpy.zeros((ny, nx)) #Create a XxY vector of zeros for omega (w)

l1_target = 1e-6

In [3]:
def L1norm(new, old):
    norm = numpy.sum(numpy.abs(new-old))
    return norm

In [12]:
def stokes(w, psi, dx, dy, u, l1_target):
    
    l1norm = 1
    psi_n = numpy.empty_like(psi)
    w_n = numpy.empty_like(w)
    iterations = 0
    
    while l1_norm > l1_target:
        
        psi_n = psi.copy()
        w_n = w.copy()
        
        psi[1:-1, 1:-1] = 1/(2*(dx**2 + dy**2)) * \
                        ((psi_n[1:-1,2:]+psi_n[1:-1,:-2])*dy**2 +\
                        (psi_n[2:,1:-1] + psi_n[:-2,1:-1])*dx**2 +\
                         w[1:-1,1:-1]*dx**2*dy**2)
        
        w[1:-1, 1:-1] = 0.25* (w_n[1:-1, 2:] + w_n[1:-1, :-2] \
                               + w_n[2:, 1:-1] + w_n[:-2, 1:-1]
                  
        
        #Dirichlet B.C. are automatically enforced
                               
        #Neumann B.C. for omega
        w[-1, 1:-1] = -1/(2*dy**2)*(8*psi[-2, 1:-1]-psi[-3, 1:-1]) - 3*u[1:-1]/dy #Top given
        w[1:-1, 0] = -1/(2*dy**2)*(8*psi[]-psi[])
        w[1:-1, -1] = -1/(2*dy**2)*(8*psi[]-psi[])
        w[0, 1:-1] = -1/(2*dy**2)*(8*psi[]-psi[])
        
            
        
        return psi, w

SyntaxError: invalid syntax (<ipython-input-12-d85e29ab963a>, line 17)

In [7]:
numpy.round(psi[32,::8], 4)

array([ 0.,  0.,  0.,  0.,  0.,  0.])